In [1]:
from scipy.optimize import root
import numpy as np
import pickle

In [2]:
with open('mf1.pkl', 'rb') as f:
    ss_params,model_params,ss_shock = pickle.load(f)


In [3]:
def assign_params(ss_params):
    a_0 = ss_params['a_0']
    a_1 = ss_params['a_1']
    a_2 = ss_params['a_2']
    alf = ss_params['alf']
    delt = ss_params['delt']
    om_0 = ss_params['om_0']
    om_1 = ss_params['om_1']
    om_2 = ss_params['om_2']
    eps = ss_params['eps']
    tht = ss_params['tht']
    bet = ss_params['bet']
    psi_k = ss_params['psi_k']
    psi_n = ss_params['psi_n']
    return eps,tht,alf,bet,om_0,om_1,om_2,delt,psi_k,psi_n,a_0,a_1,a_2


In [4]:
ss_params

{'eps': 2,
 'tht': 1,
 'alf': 0.3,
 'bet': 0.9,
 'om_0': 0.1,
 'om_1': 0.4,
 'om_2': 0.5,
 'delt': 0.05,
 'psi_n': 1,
 'psi_k': 1,
 'a_0': 0,
 'a_1': 0,
 'a_2': 0}

In [5]:
def equations(vars, params):
	n_0,n_1,n_2,s_0,s_1,s_2,k,c_0,c_1,c_2,i = vars
	eps,tht,alf,bet,om_0,om_1,om_2,delt,psi_k,psi_n,a_0,a_1,a_2 = assign_params(params)
 
	
	ph=(om_0*c_0**((eps-1)/eps)+om_1*c_1**((eps-1)/eps)+om_2*c_2**((eps-1)/eps)	)**((1-tht*eps)/(eps-1))

	v_0=om_0*c_0**(-1/eps)*ph
	v_1=om_1*c_1**(-1/eps)*ph
	v_2=om_2*c_2**(-1/eps)*ph

	q_0=(1-psi_n/2*(n_0/n_0-1)**2)
	q_1=(1-psi_n/2*(n_1/n_1-1)**2)
	q_2=(1-psi_n/2*(n_2/n_2-1)**2)


	fn_0=np.exp(a_0)*(1-alf)*(s_0/n_0/q_0)**alf*k**alf
	fn_1=np.exp(a_1)*(1-alf)*(s_1/n_1/q_1)**alf*k**alf
	fn_2=np.exp(a_2)*(1-alf)*(s_2/n_2/q_2)**alf*k**alf
 
	fs_0=np.exp(a_0)*alf*(n_0/s_0*q_0)**(1-alf)*k**alf
	fs_1=np.exp(a_1)*alf*(n_1/s_1*q_1)**(1-alf)*k**alf
	fs_2=np.exp(a_2)*alf*(n_2/s_2*q_2)**(1-alf)*k**alf
  
	x_10=fn_0*(q_0**(1-alf)-n_0*psi_n/q_0**alf/n_0*(n_0/n_0-1))
	x_11=fn_1*(q_1**(1-alf)-n_1*psi_n/q_1**alf/n_1*(n_1/n_1-1))
	x_12=fn_2*(q_2**(1-alf)-n_2*psi_n/q_2**alf/n_2*(n_2/n_2-1))

	x_20=psi_n*(n_0/n_0-1)*(n_0/n_0)**2
	x_21=psi_n*(n_1/n_1-1)*(n_1/n_1)**2
	x_22=psi_n*(n_2/n_2-1)*(n_2/n_2)**2
 
	z_1=1-psi_k*(i/k-delt)
	z_2=1-delt-psi_k/2*(i/k-delt)**2+psi_k*i/k*(i/k-delt)
 
	eq1=n_0+n_1+n_2-1
	eq2=s_0+s_1+s_2-1
	eq3=v_1*x_11*fn_1+v_1*x_21*fn_1-(v_0*x_10*fn_0+v_0*x_20*fn_0)
	eq4=v_2*x_12*fn_2+v_2*x_22*fn_2-(v_0*x_10*fn_0+v_0*x_20*fn_0)
	eq5=v_1*fs_1-v_0*fs_0
	eq6=v_2*fs_2-v_0*fs_0
	eq7=bet*(fs_0/k+z_2/z_1)-(v_0/v_0/z_1)
	eq8=(1-delt)*k+i-psi_k/2*(i/k-delt)**2*k-k
	eq9=c_0+i-np.exp(a_0)*n_0**(1-alf)*s_0**alf*k**alf
	eq10=c_1-np.exp(a_1)*n_1**(1-alf)*s_1**alf*k**alf
	eq11=c_2-np.exp(a_2)*n_2**(1-alf)*s_2**alf*k**alf
	
	return [eq1,eq2,eq3,eq4,eq5,eq6,eq7,eq8,eq9,eq10,eq11]


In [6]:
initial_guess = [.6,.2,.2,.6,.2,.2,1,.3,.3,.3,0]
def solve_equations(params):
    
    sol = root(equations, initial_guess, args=(ss_params,))
    
    # Extract the solution vector
    n_0,n_1,n_2,s_0,s_1,s_2,k,c_0,c_1,c_2,i = sol.x
    #eps,tht,alf,bet,om_0,om_1,om_2,delt,psi_k,psi_n,a_0,a_1,a_2 = assign_params(params)
    a_0=0
    a_1=0
    a_2=0
    ss_dict = {
    'n_0': n_0,
    'n_1': n_1,
    'n_2': n_2,
    's_0': s_0,
    's_1': s_1,
    's_2': s_2,    
    'k': k,
    'c_0': c_0,
    'c_1': c_1,
    'c_2': c_2,
    'i': i,
    'a_0': a_0,
    'a_1': a_1,
    'a_2': a_2
    }
    return sol, ss_dict
    

# Call the wrapper function
sol, ss_dict=solve_equations(ss_params)
print(sol)
ss_dict


 message: The iteration is not making good progress, as measured by the 
            improvement from the last ten iterations.
 success: False
  status: 5
     fun: [ 0.000e+00  0.000e+00  4.900e-01  6.533e-01  3.000e-01
            4.000e-01  1.308e-01 -5.125e-02 -3.000e-01  1.000e-01
            1.000e-01]
       x: [ 6.000e-01  2.000e-01  2.000e-01  6.000e-01  2.000e-01
            2.000e-01  1.000e+00  3.000e-01  3.000e-01  3.000e-01
            0.000e+00]
    nfev: 33
    fjac: [[-7.739e-01  0.000e+00 ...  0.000e+00  0.000e+00]
           [-2.586e-01  0.000e+00 ...  2.621e-01  0.000e+00]
           ...
           [ 2.267e-01 -1.693e-01 ...  5.740e-02  5.740e-02]
           [-6.503e-02  6.503e-02 ...  8.186e-10  1.564e-09]]
       r: [       nan        nan ...        nan        nan]
     qtf: [-2.757e-01  2.698e-01  3.003e-01 -3.225e-02 -1.171e-01
           -7.435e-02  7.176e-01  2.965e-01 -1.888e-01 -3.773e-01
            8.059e-02]


C:\Users\blankenw\AppData\Local\Temp\ipykernel_16516\3756411215.py:6: RuntimeWarning: invalid value encountered in scalar power
  ph=(om_0*c_0**((eps-1)/eps)+om_1*c_1**((eps-1)/eps)+om_2*c_2**((eps-1)/eps)	)**((1-tht*eps)/(eps-1))
C:\Users\blankenw\AppData\Local\Temp\ipykernel_16516\3756411215.py:8: RuntimeWarning: invalid value encountered in scalar power
  v_0=om_0*c_0**(-1/eps)*ph
C:\Users\blankenw\AppData\Local\Temp\ipykernel_16516\3756411215.py:44: RuntimeWarning: invalid value encountered in scalar power
  eq9=c_0+i-np.exp(a_0)*n_0**(1-alf)*s_0**alf*k**alf


{'n_0': 0.6,
 'n_1': 0.2,
 'n_2': 0.2,
 's_0': 0.6,
 's_1': 0.2,
 's_2': 0.2,
 'k': 1.0,
 'c_0': 0.3,
 'c_1': 0.3,
 'c_2': 0.3,
 'i': 0.0,
 'a_0': 0,
 'a_1': 0,
 'a_2': 0}

In [7]:
ss_dict

{'n_0': 0.6,
 'n_1': 0.2,
 'n_2': 0.2,
 's_0': 0.6,
 's_1': 0.2,
 's_2': 0.2,
 'k': 1.0,
 'c_0': 0.3,
 'c_1': 0.3,
 'c_2': 0.3,
 'i': 0.0,
 'a_0': 0,
 'a_1': 0,
 'a_2': 0}

In [8]:
# Call the wrapper function
sol, ss_dict = solve_equations(ss_params)

C:\Users\blankenw\AppData\Local\Temp\ipykernel_16516\3756411215.py:6: RuntimeWarning: invalid value encountered in scalar power
  ph=(om_0*c_0**((eps-1)/eps)+om_1*c_1**((eps-1)/eps)+om_2*c_2**((eps-1)/eps)	)**((1-tht*eps)/(eps-1))
C:\Users\blankenw\AppData\Local\Temp\ipykernel_16516\3756411215.py:8: RuntimeWarning: invalid value encountered in scalar power
  v_0=om_0*c_0**(-1/eps)*ph
C:\Users\blankenw\AppData\Local\Temp\ipykernel_16516\3756411215.py:44: RuntimeWarning: invalid value encountered in scalar power
  eq9=c_0+i-np.exp(a_0)*n_0**(1-alf)*s_0**alf*k**alf


In [9]:
# Replace values with their logarithms except for the key 'i'
for key in ss_dict:
	if  key not in ['i', 'a_0','a_1','a_2']:
		ss_dict[key] = np.log(ss_dict[key])



In [16]:
def write_dict_to_file(ss_dict, filename):
    with open(filename, 'w') as f:
        f.write('var \n')
        for key, value in ss_dict.items():
            f.write(f'{key}\n')
        f.write(f';\n\n\n')
        f.write('varexo \n')
        for key, value in ss_shock.items():
            f.write(f'{key}_shock\n')
        f.write(f';\n\n\n')
# Call the function
write_dict_to_file(ss_dict, 't_vars.m')

In [17]:
def write_dict_to_file(ss_dict, filename):
    with open(filename, 'w') as f:
        f.write('initval;\n')
        for key, value in ss_dict.items():
            f.write(f'{key}={value};\n')
        f.write(f'end;\n\n\n')        
# Call the function
write_dict_to_file(ss_dict, 't_ss.m')

In [18]:
ss_dict

{'n_0': -0.5108256237659907,
 'n_1': -1.6094379124341003,
 'n_2': -1.6094379124341003,
 's_0': -0.5108256237659907,
 's_1': -1.6094379124341003,
 's_2': -1.6094379124341003,
 'k': 0.0,
 'c_0': -1.2039728043259361,
 'c_1': -1.2039728043259361,
 'c_2': -1.2039728043259361,
 'i': 0.0,
 'a_0': 0,
 'a_1': 0,
 'a_2': 0}

In [13]:
def write_dict_to_file(params, filename):
    
    with open(filename, 'w') as f:
        f.write('parameters ')
        f.write(','.join(f'{key}' for key in params))
        f.write(';\n\n')
        for key, value in params.items():
            f.write(f'{key}={value};\n')

# Call the function
write_dict_to_file(model_params, 't_parameters.m')

In [14]:
def write_dict_to_file(ss_dict, filename):
    #variance_list = list(shock_params.values())[:3]
    with open(filename, 'w') as f:
        f.write('shocks;\n')
        i=0
        for key, value in ss_shock.items():
            f.write(f'var {key}_shock=var_{key};\n')
            i+=1
        f.write(f'\n')
        f.write(f'end;\n\n\n')        
# Call the function
write_dict_to_file(ss_dict, 't_shocks.m')

In [15]:
with open('mf2.pkl', 'wb') as f:
    pickle.dump((ss_dict), f)